cotton_bowl.ipynb

In [1]:
import os
import requests
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

In [3]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

def get_event_id():
    """Get the event ID for the Cotton Bowl game"""
    load_dotenv()
    api_key = os.getenv("ticketmaster_api_key")
    
    # Search endpoint
    url = "https://app.ticketmaster.com/discovery/v2/events.json"
    
    params = {
        "apikey": api_key,
        "keyword": "Cotton Bowl Texas Longhorns Ohio State",
        "venueId": "KovZpZAFnlEA",  # AT&T Stadium venue ID
        "size": 1
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        if '_embedded' in data and 'events' in data['_embedded']:
            event = data['_embedded']['events'][0]
            return event.get('id')
        return None
        
    except Exception as e:
        print(f"Error finding event: {e}")
        return None

def get_cotton_bowl_tickets():
    """Get all available tickets for the Cotton Bowl"""
    load_dotenv()
    api_key = os.getenv("ticketmaster_api_key")
    
    # First get the event ID
    event_id = get_event_id()
    if not event_id:
        print("Could not find the Cotton Bowl event")
        return pd.DataFrame()
    
    print(f"Found event ID: {event_id}")
    
    # Now get the ticket data
    url = f"https://app.ticketmaster.com/discovery/v2/events/{event_id}"
    
    params = {
        "apikey": api_key,
        "includePriceRanges": "true",
        "includePromotions": "true"
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        ticket_data = []
        
        # Extract seat blocks and pricing data
        if 'priceRanges' in data:
            for price_range in data['priceRanges']:
                ticket_info = {
                    'ticket_type': price_range.get('type', 'Standard'),
                    'section_type': price_range.get('area', 'General'),
                    'min_price': price_range.get('min'),
                    'max_price': price_range.get('max'),
                    'currency': price_range.get('currency'),
                }
                ticket_data.append(ticket_info)
        
        # Create DataFrame
        df = pd.DataFrame(ticket_data)
        
        # Clean up the DataFrame
        if not df.empty:
            # Sort by price
            df = df.sort_values('min_price')
            
            # Format prices to 2 decimal places
            df['min_price'] = df['min_price'].round(2)
            df['max_price'] = df['max_price'].round(2)
            
            # Add availability status
            df['status'] = 'Available'  # Default status
            
        return df
    
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err.response.status_code}")
        print(f"Error details: {err.response.text}")
        return pd.DataFrame()
    except Exception as err:
        print(f"Error occurred: {err}")
        return pd.DataFrame()

if __name__ == "__main__":
    # Get ticket DataFrame
    tickets_df = get_cotton_bowl_tickets()
    
    if not tickets_df.empty:
        print("\nCotton Bowl Tickets Available:")
        print(tickets_df)
        
        # Quick summary statistics
        print("\nSummary Statistics:")
        print(f"Total Price Ranges: {len(tickets_df)}")
        print(f"Lowest Available Price: ${tickets_df['min_price'].min():.2f}")
        print(f"Highest Available Price: ${tickets_df['max_price'].max():.2f}")
    else:
        print("No ticket data available")

Could not find the Cotton Bowl event
No ticket data available
